In [1]:
# import libraries
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

In [2]:
# following imports are required by PKI
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

In [5]:
class Client:
    def __init__(self):
        random = Crypto.Random.new().read
        # Generates the private and public keys 1024bit
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        
        # Creat signer
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        ###
        # The identity is unique to each client and can be made 
        #publicly available. Anybody would be able to send virtual 
        #currency to you using this identity and it will get added 
        #to your wallet.
        ###
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [4]:
#Testing Client
Dinesh = Client()
Dinesh.identity

'30819f300d06092a864886f70d010101050003818d00308189028181009fd8e7c11f7d5e7c2cea05f243ed92f1c5bd7586679a94f7f688dfdd843601b795b635c8950f8dd165c6be1c246210655d84df01fdb60f294b53425934b85e25da9ace060bb209e9ddb727862cbd4bb5ae924b41778fcf6ab15b36b1742591976df1f26eec95afc705f1943f03f7ed0b28b6fa2fccef66acbdfcdf5103898c730203010001'

In [10]:
class Transaction:
    #Client can be both a sender or a recipient of the money.
    #Create a transaction and specify your public address in it.
    
    def __init__(self, sender, recipient, value):
        #Initialization of a transaction class
        self.sender = sender
        self.recipient = recipient # Identity of recipient
        self.value = value
        self.time = datetime.datetime.now()
    
    def to_dict(self):
        #Combines all the four above-mentioned instance variables 
        #in a dictionary object.
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time' : self.time})
    
    def sign_transaction(self):
        #The generated signature is decoded to get the ASCII 
        #representation for printing and storing it in our blockchain.
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')
    
    def display(self):
        dict = self.to_dict()
        print ("sender: " + dict['sender'])
        print ('-----')
        print ("recipient: " + dict['recipient'])
        print ('-----')
        print ("value: " + str(dict['value']))
        print ('-----')
        print ("time: " + str(dict['time']))
        print ('-----')
    

In [11]:
#Create two users, called Dinesh and Ramesh
Dinesh = Client()
Ramesh = Client()

#Dinesh is sending payment to Ramesh, he will need 
#the public key of Ramesh which is obtained by using 
#the identity property of the client.

#Create the transaction 
t = Transaction(
   Dinesh,
   Ramesh.identity,
   5.0
)
t.display()

sender: 30819f300d06092a864886f70d010101050003818d0030818902818100b6bfc42beb9568ab13eb184455edadcf82e16aa8b179f44e51d6fb460d5e139e17168e8105bb524fd98ad86f05ef98933a4a8f3444199278b5c14781ef4a06ba7928878d98fa83c8d366fc9372c5fe00278ac6443fa00c628d7b9942096b919405698586b254081847a07ff4e90bd00c4d4bca4eebc6516d214ea3d3cbc4f7d50203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d00308189028181008f53d6461ca7cbfe52b14ffa3ba9874e455ce1b7aa65cd77c22303a36ed4c6d8de649cfc574296172f38a5eb7f3d7efbe7222b63df90d0f7971ff68dbdece4a25225ed63b532c937895a1b248e4ed002df33302b23a3bfb6c0cc11f8b4247eae0aee991253a36d3661d4cd9d3c7a7734999bd3aa99c99d7f504b38e3246a27a70203010001
-----
value: 5.0
-----
time: 2019-10-05 21:36:24.971144
-----
